In [1]:
import json
from pymongo import MongoClient
import pymongo

In [2]:
# Creating a MongoDB instance to receive and store the data from json file
client = MongoClient()
#Creating database Restaurant and table rest_detail to store the data
db = client['Restaurant']
rest_detail=db.rest_detail

#### Creating a variable to store json data

In [3]:
with open('res.json','r') as jsonfile:
    data = [json.loads(_.replace('}]}"},', '}]}"}')) for _ in jsonfile.readlines()]

In [4]:
# Converting _id value and removing '$oid' to create a corrected restaurant ID
for i in data:
    i['_id']=i['_id']['$oid']

In [5]:
# Verifying if the json file has been correctly imported into variable 'data'
data[:3]

[{'URL': 'http://www.just-eat.co.uk/restaurants-cn-chinese-cardiff/menu',
  '_id': '55f14312c7447c3da7051b26',
  'address': '228 City Road',
  'address line 2': 'Cardiff',
  'name': '.CN Chinese',
  'outcode': 'CF24',
  'postcode': '3JH',
  'rating': 5,
  'type_of_food': 'Chinese'},
 {'URL': 'http://www.just-eat.co.uk/restaurants-atthai-ss9/menu',
  '_id': '55f14312c7447c3da7051b27',
  'address': '376 Rayleigh Road',
  'address line 2': 'Essex',
  'name': '@ Thai',
  'outcode': 'SS9',
  'postcode': '5PT',
  'rating': 5.5,
  'type_of_food': 'Thai'},
 {'URL': 'http://www.just-eat.co.uk/restaurants-atthairestaurant/menu',
  '_id': '55f14312c7447c3da7051b28',
  'address': '30 Greyhound Road Hammersmith',
  'address line 2': 'London',
  'name': '@ Thai Restaurant',
  'outcode': 'W6',
  'postcode': '8NX',
  'rating': 4.5,
  'type_of_food': 'Thai'}]

#### Creating a unique id index for restaurant ID to avoid duplicate entries into MongoDB 

In [6]:
rest_detail.create_index("name", unique=True)
list(rest_detail.index_information())

['_id_', 'name_1', 'name_text']

#### Creating an index for text search to be used in Case 2

In [7]:
rest_detail.create_index([("name",pymongo.TEXT)])

'name_text'

#### Inserting data into mongoDB table rest_detail for rest_id, address, zipcode = outcode+postcode,type of food and rating of the restaurant, removing URL attribute

In [8]:
for i in data:
    try:
        query={'rest_id':i['_id'],"name":i["name"],"address":i['address'],'address_line2':i['address line 2'],
               'zipcode':i['outcode']+i['postcode'],'type_of_food':i['type_of_food'],'rating':i['rating']}
        rest_detail.insert_one(query)
    except Exception as e:
        pass

In [9]:
# To verify if the records have been correctly inserted into mongoDB
rest_detail.find_one()

{'_id': ObjectId('6148c7cf655656e743a82ddc'),
 'rest_id': '55f14312c7447c3da7051b26',
 'name': '.CN Chinese',
 'address': '228 City Road',
 'address_line2': 'Cardiff',
 'zipcode': 'CF243JH',
 'type_of_food': 'Chinese',
 'rating': 5}

### Verifying if the unique index created on 'name' is restricting any duplicate value insertion with an existing name '042 Restaurant & Bar'

In [10]:
try:
    print("Trying to insert values for an existing rest_id: 55f14312c7447c3da7051b26"+'\n')
    query={'rest_id':'55f14312c7447c3da7051b26',"name":'042 Restaurant & Bar',"address":'City Road','address_line2':'Cardiff',
           'zipcode':'E111HR','type_of_food':'Chinese','rating':7}
    rest_detail.insert_one(query)
except Exception as e:
    print("Exception", e.__class__, "occurred for a duplicate insertion!")

Trying to insert values for an existing rest_id: 55f14312c7447c3da7051b26

Exception <class 'pymongo.errors.DuplicateKeyError'> occurred for a duplicate insertion!


### As evident from the error message above, duplicate entries are not allowed for an existing rest_id in the table